In [7]:
import psycopg2
import configparser
import os
import pandas as pd
from sqlalchemy import create_engine, text
import subprocess
import sys
import papermill as pm
import json
import math
from psycopg2.extras import execute_batch
def get_db_config():
    config = configparser.ConfigParser()
    config.read('../api_data/db.ini')
    
    return {
        'database': config['postgresql']['database'],
        'user': config['postgresql']['user'],
        'password': config['postgresql']['password'],
        'host': config['postgresql']['host'],
        'port': config['postgresql']['port']
    }
# connect to database 
db_config = get_db_config()
conn = psycopg2.connect(
    database=db_config['database'],
    user=db_config['user'],
    password=db_config['password'],
    host=db_config['host'],
    port=db_config['port']
)
cursor = conn.cursor()
print("Database connected successfully")

Database connected successfully


In [9]:
# Create SQLAlchemy engine for pandas to_sql functionality
CONNECTION_STR = (
    f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['database']}"
)
ENGINE = create_engine(CONNECTION_STR)

def num_of_possessions_down():
    db_query = """
        SELECT game_id, eventnum, scoremargin 
        FROM play_by_play_q4 
        WHERE scoremargin IS NOT NULL 
        ORDER BY eventnum ASC
    """
    possessions_db = pd.read_sql(db_query, ENGINE)

    updates = []
    for _, play in possessions_db.iterrows():
        score_margin = play['scoremargin']
        possessions = 0
        if 'TIE' not in str(score_margin):
            possessions = math.ceil(abs(int(score_margin)) / 3)
        updates.append((possessions, play['game_id'], play['eventnum']))

    conn = ENGINE.raw_connection()
    cursor = conn.cursor()

    update_query = """
        UPDATE play_by_play_q4
        SET possessions_lead_or_trail = %s
        WHERE game_id = %s AND eventnum = %s
    """
    execute_batch(cursor, update_query, updates, page_size=1000)
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Updated {len(updates)} rows successfully.")
    
num_of_possessions_down()
    



Updated 453803 rows successfully.
